# Computer Vision on the Descartes Labs Platform - Deploying a Segmentation Model
__________________

This notebook will demonstrate how one can utilize Descartes Labs Python APIs to efficiently prototype and iterate on deploying, or running inference, an image segmentation model over large areas of interest (AOIs). This is meant to serve _solely as a jumping off point_ and is not intended to be used as a panacea for all machine learning needs. 

The general outline of this sample is as follows:
* Create a new [`Product`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/product.html) with a single [`Band`](https://docs.descarteslabs.com/descarteslabs/catalog/readme.html) to store the results of the model trained in [03b Training a Segmentation Model.ipynb](03b%20Training%20a%20Segmentation%20Model.ipynb)
* Define an asynchronous [`Function`](https://docs.descarteslabs.com/descarteslabs/compute/readme.html#descarteslabs.compute.Function) which will:
    * Accept a single [`DLTile`](https://docs.descarteslabs.com/descarteslabs/geo/readme.html#descarteslabs.geo.DLTile) key and output product ID as an input argument
    * Retrieve the segmentation model weights file stored as a [`Blob`'](https://docs.descarteslabs.com/descarteslabs/catalog/docs/blob.html#descarteslabs.catalog.Blob)
    * Search and retrieve the corresponding [National Agricultural Imagery Program (NAIP)](https://app.descarteslabs.com/explorer/datasets/usda:naip:v1) imagery and _infer_ the model
    * Save the results of the model's deployment back to a new [`Image`](https://docs.descarteslabs.com/descarteslabs/catalog/docs/image.html)
* Explore options for defining large AOIs to deploy this model:
    * Arbitrary geometries
    * Optionally through an interactive [`Dynamic Compute`](https://docs.descarteslabs.com/api/dynamic-compute.html) map in [03d Interactive Deployment with Dynamic Compute.ipynb](03d%20Interactive%20Deployment%20with%20Dynamic%20Compute.ipynb)
    
    
#### Note: After completing [03b Training a Segmentation Model.ipynb](03b%20Training%20a%20Segmentation%20Model.ipynb) it is advised to shut down the kernel to avoid memory limits

In [ ]:
import descarteslabs as dl
from descarteslabs.catalog import (
    GenericBand,
    Blob,
    Image,
    OverviewResampler,
    Product,
    properties as p,
)
from descarteslabs.compute import Function
from descarteslabs.vector import Table

In [ ]:
import sys
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf

Setting global variables:

In [ ]:
func_name = "Deploy Wellpad Model"

In [ ]:
user_hash = dl.auth.Auth().namespace
org = dl.auth.Auth().payload['org']

In [ ]:
major = sys.version_info.major
minor = sys.version_info.minor
compute_image = f"python{major}.{minor}:latest"
compute_image

## Creating an Output Product
Below we create a new product to save our model results to. Note we are appending our user ID to the end, but in practice this is not required _as long as your ID is unique to your organization_.

In [ ]:
res_pid = f"segmentation-outputs-{user_hash}"

#### Note on Product ID Creation
Since this is an example, where multiple users at the same organization may re-run this notebook, we intend to delete and overwrite the output product upon every iteration. Because of this we must reconstruct the product's _namespace_. In the following cell we will append either the current user's _organization_ or _user hash_ to the passed product ID:

In [ ]:
try:
    assert org
    res_pid = f"{org}:{res_pid}"
except:
    res_pid = f"{user_hash}:{res_pid}"
res_pid

We do not always need to delete and overwrite our product on every iteration as in the following cell. This notebook is designed for demonstration purposes, where we do not care about preserving each prior product.

In practice, as long as your product has a **unique** ID you may ignore the next cell and skip to the following.

In [ ]:
result_product = Product.get_or_create(res_pid)

if result_product.state == dl.catalog.DocumentState.SAVED:
    status = result_product.delete_related_objects()
    if status:
        status.wait_for_completion()
    result_product.delete()

In [ ]:
res_product = Product.get_or_create(res_pid)
res_product.name = "Testing Segmentation Outputs"
res_product.tags = ["examples"]
res_product.readers = []
res_product.save()
res_product

Creating our output band:

In [ ]:
band = GenericBand.get_or_create(
    id=f"{res_product.id}:class",
    band_index=0,
    data_type=dl.catalog.DataType.FLOAT32,
    data_range=[0, 1],
    display_range=[0, 1],
    nodata=0,
    colormap_name="viridis",
    resolution=dl.catalog.Resolution(value=1.0, unit=dl.catalog.ResolutionUnit.METERS),
)
band.save()

## Scaling with Batch Compute
Here we define a local function to send to our compute service which:
* Accepts a tile key and product ID to write to
* Retrieves the model 
* Searches imagery over our tile
* Infers, or runs, our model over the rastered imagery
* Write the results back to our product as a new image

In [ ]:
def write_segmentation_to_catalog(dltile_key, out_pid):
    import descarteslabs as dl
    import numpy as np
    import os
    from descarteslabs.catalog import (
        Blob,
        Product,
        Image,
        OverviewResampler,
        properties as p,
    )

    from keras.models import load_model

    org = dl.auth.Auth().payload["org"]
    user_id = dl.auth.Auth().namespace

    print("Starting process...")
    blob = Blob.get(name="training_segmentation")
    blob.download("segmentation_model.keras")
    print("Downloaded model...")

    model = load_model(f"segmentation_model.keras")
    print("Loaded model...")
    # Getting DLTile, finding input Images
    dltile = dl.geo.DLTile.from_key(dltile_key)

    naip_pid = "usda:naip:v1"
    bands = ["nir", "red", "green"]

    naip_prod = Product.get(naip_pid)

    naip_ic = (
        naip_prod.images()
        .intersects(dltile)
        .filter("2016-01-01" < p.acquired < "2017-01-01")
    ).collect()
    print("Searched imagery...")
    arr = naip_ic.mosaic(bands, bands_axis=-1)
    print("Retrieved imagery...")
    preds = model.predict(np.array([arr]))[0, :, :, 0]
    # Masking out very low values
    preds[preds < 0.1] = 0
    print("Complete predictions...")

    out_product = dl.catalog.Product.get(out_pid)
    print(f"Writing to {out_product.id}")
    # Creating an image - note the required unique id corresponding to the DLTile
    image = Image(
        product=out_product,
        id=f"{out_product.id}:{dltile_key.replace(':', '_')}",
    )
    print("Writing image...")
    # Setting image geotransform + projection from dltile info
    image.geotrans = dltile.geotrans
    image.projection = dltile.proj4
    image.acquired = "2023-11-28"  # Make sure this is accurate
    image.extra_properties = {"foo": "bar"}  # You can add up to 50 extra props
    upload = image.upload_ndarray(
        ndarray=preds,
        overviews=[2, 4, 8, 16, 32, 64],
        overview_resampler=OverviewResampler.NEAREST,
        overwrite=True,
    )
    upload.wait_for_completion()
    print("Cleaning up...")
    os.remove("segmentation_model.keras")
    return image.id

Sample tile:

In [ ]:
dltile = dl.geo.DLTile.from_latlon(
    33.4730, -101.4974, resolution=1.0, tilesize=512, pad=0
)

Sample iteration of our function, locally:

In [ ]:
img_id = write_segmentation_to_catalog(dltile.key, res_product.id)
out_img = Image.get(img_id)
ndarr = out_img.ndarray("class")
plt.imshow(ndarr[0])

Defining the compute function:

In [ ]:
async_func = Function(
    write_segmentation_to_catalog,
    name=func_name,
    image=compute_image,
    cpus=1,
    memory=2,
    timeout=100,
    maximum_concurrency=50,
    retry_count=0,
    requirements=[
        f"tensorflow=={tf.__version__}",
    ],
)
async_func.save()
print(f"Saved {async_func.id}")

Defining an AOI and splitting into tiles:

In [ ]:
geom = [
    {
        "geometry": {
            "coordinates": [
                [
                    [-101.5687330486888, 33.46344873260057],
                    [-101.57138184506375, 33.54154310531996],
                    [-101.4349688317516, 33.54233158762628],
                    [-101.43477963201026, 33.46123897849522],
                    [-101.5687330486888, 33.46344873260057],
                ]
            ],
            "type": "Polygon",
        },
    }
]
dltiles = dl.geo.DLTile.from_shape(geom, resolution=1.0, tilesize=512, pad=0)
len(dltiles)

And a set of input arguments:

In [ ]:
args = [(dltile.key, res_product.id) for dltile in dltiles]

And finally mapping our arguments:

In [ ]:
jobs = async_func.map(args)
len(jobs)

At this point we can wait asynchronously via:

    async_func.wait_for_completion()

Or navigate to [app.descarteslabs.com/compute](https://app.descarteslabs.com/compute) to track and manage your function's progress.

#### *Optional Vectorization Example:*
Example of vectorizing outputs, with a given threshold:

In [ ]:
from rasterio.features import shapes
from rasterio.transform import Affine
from rasterio.plot import reshape_as_raster
from shapely.geometry import shape

In [ ]:
trans = Affine.from_gdal(*dltile.geotrans)

In [ ]:
thresh = 0.8
ndarr[ndarr > thresh] = 1

In [ ]:
polys = list(shapes(ndarr, mask=(ndarr >= 1), transform=trans))
poly_list = [shape(poly[0]) for poly in polys]

In [ ]:
vector_gdf = gpd.GeoDataFrame({"geometry": poly_list}, crs=dltile.crs).to_crs(
    dltile.crs
)
vector_gdf.plot()